![Logo](./1.png)

Pour voir **les points de montage** actifs sur ton **nœud Proxmox**, tu peux utiliser plusieurs commandes qui te donneront une vue d'ensemble des systèmes de fichiers montés.

### 1. **Commandes pour voir les points de montage**

#### 1.1 **Commande `mount`**
La commande `mount` te montre tous les systèmes de fichiers actuellement montés, y compris les points de montage.

Ouvre un terminal sur ton serveur Proxmox et tape la commande suivante :

```bash
mount
```

Cela te donnera une liste de tous les points de montage sur ton nœud, par exemple :

```
/dev/sda1 on / type ext4 (rw,relatime)
/dev/sdb1 on /mnt/partage type ext4 (rw,relatime)
...
```

- **/dev/sda1** : le périphérique de stockage (par exemple, un disque).
- **/mnt/partage** : le **point de montage** où ce périphérique est accessible.
  
Cette commande te montre aussi les options de montage (par exemple, `rw` pour lecture/écriture).

#### 1.2 **Commande `lsblk`**
La commande `lsblk` liste tous les périphériques de stockage (disques, partitions) et leurs points de montage associés.

```bash
lsblk
```

Cela donnera une sortie similaire à ceci :

```
NAME   MAJ:MIN RM   SIZE RO TYPE MOUNTPOINT
sda      8:0    0   100G  0 disk 
├─sda1   8:1    0   100G  0 part /
sdb      8:16   0   500G  0 disk 
└─sdb1   8:17   0   500G  0 part /mnt/partage
```

- **MOUNTPOINT** montre où chaque disque ou partition est monté. Dans cet exemple, `/mnt/partage` est le point de montage du disque `sdb1`.

#### 1.3 **Commande `df`**
La commande `df` est utilisée pour afficher l’espace disque, mais elle montre aussi les points de montage. 

```bash
df -h
```

Cela te donnera une sortie similaire à ceci :

```
Filesystem      Size  Used Avail Use% Mounted on
/dev/sda1       100G  20G   80G  20% /
/dev/sdb1       500G  50G  450G  10% /mnt/partage
```

- **Mounted on** : affiche le **point de montage** de chaque système de fichiers.

### 2. **Vérifier les points de montage des conteneurs LXC**

Si tu veux spécifiquement vérifier quels points de montage sont configurés pour tes **conteneurs LXC** sur Proxmox, il n'y a pas de commande unique pour lister tous les points de montage, mais tu peux vérifier chaque fichier de configuration des conteneurs.

#### 2.1 **Vérifier le fichier de configuration d’un conteneur LXC**
Les fichiers de configuration des conteneurs LXC se trouvent dans `/etc/pve/lxc/` et sont nommés par l'ID du conteneur, par exemple `101.conf` pour le conteneur 101. Ces fichiers contiennent les informations sur les points de montage configurés pour chaque conteneur.

Tu peux ouvrir et vérifier le contenu des fichiers `.conf` des conteneurs pour voir les points de montage.

```bash
cat /etc/pve/lxc/101.conf
```

Dans ce fichier, tu pourrais voir des lignes comme :

```bash
mp0: /mnt/partage,mp=/mnt/container_partage
```

Cela montre que le répertoire `/mnt/partage` de l’hôte est monté sur `/mnt/container_partage` à l’intérieur du conteneur.

---

### 3. **Résumé des commandes :**

- `mount` : liste tous les points de montage actifs, y compris ceux du système et des périphériques externes.
- `lsblk` : montre tous les périphériques de stockage et leurs points de montage.
- `df -h` : montre l’espace disque et les points de montage.
- **Vérification des conteneurs LXC** : consulte les fichiers de configuration dans `/etc/pve/lxc/` pour voir les points de montage spécifiques aux conteneurs.

Ces commandes te permettront de voir tous les points de montage et de mieux comprendre comment ton système et tes conteneurs LXC accèdent aux fichiers.

Si tu as des questions supplémentaires ou si tu veux explorer un détail en particulier, n'hésite pas à demander ! 😄

### 1. ✅ **Est-ce que le point de montage est défini dans la config du conteneur ?**

Sur ton nœud, vérifie la config de ton conteneur nginx (par exemple ID `101`) :

```bash
cat /etc/pve/lxc/101.conf
```

Tu devrais voir une ligne comme :

```bash
mp0: /mnt/data,mp=/mnt/data
```

- **À gauche de la virgule** : le chemin sur le **nœud Proxmox**.
- **À droite de la virgule** : le chemin où ce dossier sera visible dans le **conteneur**.

➡️ **S’il n’y a pas cette ligne**, alors ton conteneur a juste un dossier "local" `/mnt/data` qui n’a **aucune connexion** avec le nœud. C’est pour ça que les fichiers n’apparaissent pas.

---

### 2. ✅ **Redémarrer le conteneur après modification**

Une fois que tu ajoutes la ligne `mp0` dans `103.conf`, tu dois **redémarrer le conteneur** pour que le montage soit actif :

```bash
pct reboot 101
```

---

## 📌 En résumé

| Vérification | Description |
|--------------|-------------|
| Config `mp0` | Est-ce que le point de montage est bien déclaré dans le fichier `.conf` du conteneur ? |
| Redémarrage | As-tu redémarré le conteneur après ajout du `mp0` ? |
| Dossier cible | Est-ce que `/mnt/data` existe sur le nœud ? |
| Permissions | Le conteneur peut-il écrire dedans ? (UID mapping, chmod) |

---

Parfait, alors on va vérifier la configuration du conteneur **LXC 101** (celui de `nginx`) pour s'assurer que le **point de montage `/mnt/data`** est bien en place.

### ✅ 1. Regarde le contenu du fichier de config du conteneur :

Sur ton **nœud Proxmox**, exécute cette commande :

```bash
cat /etc/pve/lxc/101.conf
```

Et cherche une ligne qui ressemble à ceci :

```bash
mp0: /mnt/data,mp=/mnt/data
```

- Si **la ligne est là**, alors tout est bon du côté config.
- Si **elle est absente**, ça veut dire que **le conteneur ne partage pas réellement** le dossier `/mnt/data` avec le nœud. Il utilise juste un dossier local au conteneur.

---

### ✏️ 2. Si la ligne est absente, ajoute-la manuellement

Toujours sur le **nœud Proxmox**, ouvre le fichier de config avec nano (ou vim si tu préfères) :

```bash
nano /etc/pve/lxc/101.conf
```

Et ajoute tout en bas la ligne suivante :

```bash
mp0: /mnt/data,mp=/mnt/data
```

(Si tu veux utiliser un autre chemin dans le conteneur, tu peux changer `mp=/mnt/data` en ce que tu veux, genre `/srv/partage`.)

Ensuite, **sauvegarde et quitte** :  
- `Ctrl + O` pour enregistrer  
- `Ctrl + X` pour quitter

---

### 🔄 3. Redémarre le conteneur

Pour que le montage soit pris en compte, redémarre ton conteneur :

```bash
pct restart 101
```

---

### 🔁 4. Vérifie si le partage fonctionne

Dans ton conteneur (depuis le nœud, tape) :

```bash
pct enter 101
```

Puis :

```bash
touch /mnt/data/test_nginx.txt
ls /mnt/data
```

Et ensuite, sur le **nœud Proxmox** :

```bash
ls /mnt/data
```

Tu dois voir le fichier `test_nginx.txt`. 🎉

---

### ❗ Si ça ne marche toujours pas

Dis-moi ce que retourne :

```bash
cat /etc/pve/lxc/101.conf
```

Et on regarde ensemble s’il y a une erreur de chemin, de syntaxe, ou de droits. Tu peux copier-coller ici le contenu du fichier, et je t’aide ligne par ligne.

Tu veux qu'on le fasse ensemble ?

Dans le conteneur :

```bash
mount | grep /mnt/data
```

Tu devrais voir une ligne du genre :

```
/mnt/video/lxc-data-100 on /mnt/data type none (rw,bind)
```

Si tu ne vois rien : ➡️ **le point de montage n’est pas actif**.